# Computer Vision

In [ ]:
#poniższe linijki tylko dla Colaba

# from google.colab import drive
# drive.mount('/content/drive/')


In [ ]:
#poniższa linijki tylko dla Colaba

# !ls -la '/content/drive/My Drive/Colab Notebooks/'

In [ ]:
import cv2
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt

#poniższa linijka tylko dla Colaba

# from google.colab.patches import cv2_imshow

#proszę poniżej wprowadzić odpowiednie nazwy pliku ze zdjęciem!
img_org = cv2.imread('test.png')
img2 = cv2.imread('test2.png')


In [ ]:

plt.imshow(img_org)

In [ ]:
import numpy as np
img = cv2.cvtColor(img_org, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
plt.imshow(img2)

# Filtry

In [ ]:
#Filters
import numpy as np

#Funkcja pomocnicza do stosowania filtrów
def apply_filter(source, kernel, gray=False):
    new_image = cv2.filter2D(source, -1, kernel)
    if gray:
        plt.imshow(new_image, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(new_image)
    return new_image

In [ ]:
#idendity
# proszę zadeklarować poprawny filtr identity (identyczność) jako macierz z numpy
idendity_kernel = None

print("identity")
idt = apply_filter(img, idendity_kernel)

In [ ]:
#blur
# proszę zadeklarować poprawny filtr blur jako macierz z numpy
blur_kernel = None

print("blur")
blur = apply_filter(img, blur_kernel)

In [ ]:
#sharpen
# proszę zadeklarować poprawny filtr sharpen jako macierz z numpy
sharpen_kernel = None

print("sharpen")
sharpen = apply_filter(img, sharpen_kernel)

In [ ]:
#zapisanie zdjęcia w skali szarości
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray_img, cmap=plt.get_cmap('gray'))

In [ ]:
#simply_edge_detection
# proszę zadeklarować poprawny filtr prostej detekcji krawędzi jako macierz z numpy
simply_edge_kernel = None

print("simply_edge_detection")
simple = apply_filter(gray_img, simply_edge_kernel, gray=True)

In [ ]:
#bottom sobel
# proszę zadeklarować poprawny filtr bottom sobel jako macierz z numpy
bottom_sobel_kernel = None

print("bottom_sobel")
bottom_sobel_img = apply_filter(gray_img, bottom_sobel_kernel, gray=True)

In [ ]:
#left sobel
# proszę zadeklarować poprawny filtr left sobel jako macierz z numpy
left_sobel_kernel = None

print("left_sobel")
left_sobel_img = apply_filter(gray_img, left_sobel_kernel, gray=True)

In [ ]:
#Zastosowanie full sobel
full_sobel_img = left_sobel_img / 2 + bottom_sobel_img / 2
print("full sobel")
plt.imshow(full_sobel_img, cmap=plt.get_cmap('gray'))

# Histogram Equalization

In [ ]:
import matplotlib.pyplot as plt

def show_histogram(image):
    ax1 = plt.subplot(331)
    plt.xlim([0, 256])
    plt.ylim([0, 4000])
    for i, col in enumerate(['b', 'g', 'r']):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        ax1 = plt.subplot(3,1,i+1, sharey=ax1)
        plt.plot(hist, color = col)
        
        if i<2:
            plt.setp(ax1.get_xticklabels(), visible=False)
    plt.show()

show_histogram(img)

In [ ]:
#Funkcja do wyrównywania histogramu
def show_hsv_equalized(image):
    new_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(new_image)
    eq_H = cv2.equalizeHist(H)
    eq_S = cv2.equalizeHist(S)
    eq_V = cv2.equalizeHist(V)
    eq_image = cv2.cvtColor(cv2.merge([H, S, eq_V]), cv2.COLOR_HSV2BGR)
    plt.imshow(eq_image)
    plt.show()
    show_histogram(eq_image)

show_hsv_equalized(img)

In [ ]:
show_histogram(img2)

In [ ]:
show_hsv_equalized(img2)

# Progression

In [ ]:
def threshold(image):
    @interact(x=widgets.IntSlider(min=0, max=255, step=1, value=100))
    def trackbar(x):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _,thresh1 = cv2.threshold(gray_img,x,255,cv2.THRESH_BINARY)
        plt.imshow(thresh1, cmap=plt.get_cmap('gray'))

threshold(img)

In [ ]:
def adaptive_threshold(image):
    @interact(x=widgets.IntSlider(min=3, max=15, step=2, value=9), y=True)
    def trackbar(x, y):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if y:
            adaptive_method = cv2.ADAPTIVE_THRESH_MEAN_C
        else:
            adaptive_method = cv2.ADAPTIVE_THRESH_GAUSSIAN_C
        result = cv2.adaptiveThreshold(gray_img,255,adaptive_method, 
                                        cv2.THRESH_BINARY,x,2)
        plt.imshow(result, cmap=plt.get_cmap('gray'))
        
adaptive_threshold(img)

In [ ]:
help(cv2.adaptiveThreshold)

# Morphological operation

In [ ]:
def morphological_operation(image):

    def opening(img, kernel):
        return cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    
    def closing(img, kernel):
        return cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    @interact(operation=['Erosion','Dilation', 'Opening', 'Closing'], 
                struct_el=['MORPH_RECT','MORPH_ELLIPSE', 'MORPH_CROSS'], 
                size = widgets.IntSlider(min=3, max=15, step=2, value=5))
    def trackbar(operation, struct_el, size):
        operation_dict = {'Erosion': cv2.erode,'Dilation': cv2.dilate,
                        'Opening': opening, 'Closing': closing}
        struct_el_dict = {'MORPH_RECT': cv2.getStructuringElement(cv2.MORPH_RECT,(size, size)),
                        'MORPH_ELLIPSE': cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(size, size)),
                        'MORPH_CROSS': cv2.getStructuringElement(cv2.MORPH_CROSS,(size, size))}
        result = operation_dict[operation](image, struct_el_dict[struct_el])
        plt.imshow(result, cmap=plt.get_cmap('gray'))
        
        
#using sobel output form above
full_sobel_img[full_sobel_img>30] = 255
plt.imshow(full_sobel_img, cmap=plt.get_cmap('gray'))
morphological_operation(full_sobel_img)


# Feature Extraction

In [ ]:
import numpy as np
shapes = cv2.imread('contours.png')

gray_shapes = cv2.cvtColor(shapes, cv2.COLOR_BGR2GRAY)
_,thresh1 = cv2.threshold(gray_shapes,90,255,cv2.THRESH_BINARY)
plt.imshow(thresh1, cmap=plt.get_cmap('gray'))

In [ ]:
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img_with_contours = cv2.drawContours(shapes, contours, -1, (255,0,0), 3)
plt.imshow(img_with_contours)
print(hierarchy)

In [ ]:
# Get rid of internal contours, extract information about childs

filtered_contours = []
has_a_child = []

for contour, h in zip(contours, hierarchy[0]):
    if h[3] == -1:
        filtered_contours.append(contour)
        if h[2] != -1:
            has_a_child.append(1)
        else:
            has_a_child.append(0)
print(has_a_child)

In [ ]:
# Get all usefull information from contours
features_list = []

for contour, childs in zip(filtered_contours, has_a_child):
    print(contour)
    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments)
    
    contour_area = cv2.contourArea(contour)
    arc_length = cv2.arcLength(contour, True)
    if arc_length > 0:
        features = {f'Hu moment no. {x}': hu[0] for x, hu in enumerate(hu_moments)}
        features['circularity'] = 4 * 3.14 * contour_area / arc_length**2
        features['has_child'] = childs
        features_list.append(features)

In [ ]:
# Save data as pd.DataFrame
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format
df = pd.DataFrame(features_list)
df

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

from sklearn import preprocessing

#proszę zamienić DataFrame na macierz
x = None

#zainicjalizować Normalizer i zastosować go do macierzy x
normalizer = None
x_scaled = None

#proszę wytrenować KMeans z liczbą klastrów 5 (można też z 4) na x_scaled
kmeans = None


colors = [(255,0,0), (255,255,0), (255,0,255), (0,255,0), (0,255,255), (0,0,255)]


for contour, label in zip(filtered_contours, kmeans.labels_):
    classified = cv2.drawContours(shapes, [contour], 0, colors[label], 3)


plt.imshow(classified)


# Sesja samodzielna

**MNIST**

Zbiór danych MNIST to akronim oznaczający zbiór danych Modified National Institute of Standards and Technology.

Jest to zbiór danych zawierający 60000 małych kwadratowych obrazów 28 × 28 pikseli w skali szarości, składających się z odręcznych pojedynczych cyfr od 0 do 9.

Zadanie polega na zaklasyfikowaniu danego obrazu odręcznej cyfry do jednej z 10 klas reprezentujących wartości całkowite od 0 do 9 włącznie.

Jest to powszechnie używany i głęboko rozumiany zbiór danych, który w większości jest „rozwiązany”. Modele o najwyższej wydajności to splotowe sieci neuronowe do głębokiego uczenia, które osiągają dokładność klasyfikacji powyżej 99%, przy współczynniku błędów między 0,4% a 0,2% w przypadku zestawu danych testowych.

**Zadanie**

Z wykorzystaniem konwencjonalnych metod, stwórz pipeline wydobywający cechy z obrazu i na ich podstawie spróbuj rozwiązać to zadanie.


1.   Załaduj bazę obrazów do pamieci
2.   Wyświetl kilka przykładowych obrazów, żeby zapoznać się z MNISTem
3.   Stwórz funkcję pobierającą informację o odpowiednim konturze z obrazka
4.   Stwórz funkcję pobierającą featury z konturu
5.   Przedstaw dane w postaci dataframów (nie zapomnij - zarówno dla obrazów testowych i treningowych)
6.   Znormalizuj dane!
7.   Klasyfikacja! Wyniki przedstaw w raporcie


# Zbiór odpowiedzi

1.   Załaduj bazę obrazów do pamieci




In [ ]:
from keras.datasets import mnist

#załadowanie danych mnist
_ = mnist.load_data()

#odpowiednio zadeklarować zmianne i sprawdzić ich wielkość

print('X_train: ' + str(None))
print('Y_train: ' + str(None))
print('X_test:  '  + str(None))
print('Y_test:  '  + str(None))

2.   Wyświetl kilka przykładowych obrazów, żeby zapoznać się z MNISTem


In [ ]:
#poniżej w miejscu None wyciągnąć 9 pierwszych obrazków ze zbioru treningowego
for i in range(9):  
    plt.subplot(330 + 1 + i)
    plt.imshow(None[i], cmap=plt.get_cmap('copper'))
    plt.show()


3.   Stwórz funkcję pobierającą informację o odpowiednim konturze z obrazka


In [ ]:
import pandas as pd

#funkcja do zczytania konturów i wybrania tylko "rodziców" a odrzucenia "dzieci"
def get_contour(image):
    #threshold
    _,thresh1 = cv2.threshold(image,50,255,cv2.THRESH_BINARY)
    #wyciągnięcie konturów
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour, h in zip(contours, hierarchy[0]):
        if h[3] == -1:
            return contour, len(contours)-1

4.   Stwórz funkcję pobierającą featury z konturu

In [ ]:
def get_fetures(cnt, childs):

    moments = cv2.moments(cnt)
    hu_moments = cv2.HuMoments(moments)
    features = {f'Hu moment no. {x}': hu[0] for x, hu in enumerate(hu_moments)}
    contour_area = cv2.contourArea(cnt)
    arc_length = cv2.arcLength(cnt, True)
    try:
        features['circularity'] = 4 * 3.14 * contour_area / arc_length**2
    except ZeroDivisionError:
        features['circularity'] = 0
    features['childs'] = childs
    
    return features

5.   Przedstaw dane w postaci dataframów (nie zapomnij - zarówno dla obrazów testowych i treningowych)

In [ ]:
# train data

lst_features = []
for image in train_X:  
    contour, childs = get_contour(image)
    lst_features.append(get_fetures(contour, childs))

pd.options.display.float_format = "{:,.2f}".format
df_train = pd.DataFrame(lst_features)
df_train

# test data

lst_features = []
for image in test_X:  
    contour, childs = get_contour(image)
    lst_features.append(get_fetures(contour, childs))

pd.options.display.float_format = "{:,.2f}".format
df_test = pd.DataFrame(lst_features)
df_test


6.   Znormalizuj dane!

In [ ]:
#Normalization
from sklearn import preprocessing

def normalize(data):
    #utwórz macierz na podstawie danych
    x = None
    #znormalizuj dane x i przypisz do x_scaled
    normalizer = None
    x_scaled = None
    #przypisz x_scaled do DataFrame
    data = None
    return data

df_test = normalize(df_test)
df_train = normalize(df_train)

7.   Klasyfikacja! Wyniki przedstaw w raporcie

In [ ]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

#proszę wytrenować klasyfikator. Może być dowolny - ja proponuję prosty Decision Tree Classifier
clf = None

#wytrenowanie

#przypisanie predykcji dla zbioru testowego
prediction = None

print(classification_report(test_y, prediction))